In [1]:
from fastlane_bot import Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.helpers import TxHelper
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonBot))
from fastlane_bot.testing import *
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("2.0", __VERSION__)

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
Error adding Ethereum blockchain to database Ethereum, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "ix_blockchains_name"
DETAIL:  Key (name)=(Ethereum) already exists.

[SQL: INSERT INTO blockchains (name, block_number) VALUES (%(name)s, %(block_number)s) RETURNING blockchains.id]
[parameters: {'name': 'Ethereum', 'block_number': 0}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) skipping...
Error adding exchange carbon_v1 to database, (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "exchanges_pkey"
DETAIL:  Key (id)=(6) already exists.

[SQL: INSERT INTO exchanges (id, name, blockchain_name) VALUES (%(id)s, %(name)s, %(blockchain_name)s)]
[parameters: {'id': 6, 'name': 'carbon_v1', 'blockchain_name': 'Ethereum'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj) ski

C:\Users\Kveen\AppData\Local\Temp\ipykernel_28604\1582963263.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


Version = 3-b2.1 [requirements >= 2.0 is met]


# Testing the _run functions on MAINNET [NBTest021]

## MAINNET Configuration

### Set up the bot and curves

In [2]:
C = Config.new(config=Config.CONFIG_MAINNET)

# add_tenderly = f'brownie networks add "Ethereum" "{C.NETWORK_ID}" host="{C.provider_url}" name="{C.network_name}" chainid={C.CHAIN_ID}'
# mod_tenderly = f'brownie networks modify "{C.network_id}" host="{C.provider_url}" name="{C.network_name}" chainid={C.CHAIN_ID}'
# set_tenderly = f'brownie networks set_provider "{C.network_id}" host="{C.provider_url}" name="{C.network_name}" chainid={C.CHAIN_ID}'
#
# print(add_tenderly, mod_tenderly, set_tenderly)

assert C.DATABASE == C.DATABASE_POSTGRES
assert C.POSTGRES_DB == "defaultdb"
assert C.NETWORK == C.NETWORK_MAINNET
assert C.PROVIDER == C.PROVIDER_ALCHEMY
bot = CarbonBot(ConfigObj=C)
assert str(type(bot.db)) == "<class 'fastlane_bot.db.manager.DatabaseManager'>"

In [3]:
CCm = bot.get_curves()
exch = {c.P("exchange") for c in CCm}
print("Number of curves:", len(CCm))
print("Number of tokens:", len(CCm.tokens()))
print("Exchanges:", exch)

Number of curves: 92
Number of tokens: 23
Exchanges: {'uniswap_v3', 'bancor_v3', 'carbon_v1', 'uniswap_v2'}


In [4]:
bad_curves = ['340282366920938463463374607431768211527-0',
              '340282366920938463463374607431768211527-1',
              '12930729942995661611608235082407192035404-0',
              '9868188640707215440437863615521278132277-1',
                # '2381976568446569244243622252022377480199-0',
#               '3743106036130323098097120681749450326030-0',
#               '4423670769972200025023869896612986748950-0'
]
subcurves = [x for x in CCm if x.cid not in bad_curves]
CCm2 = CPCContainer([x for x in subcurves if x.params.exchange not in ['bancor_v2']])
len(CCm2)

91

## Actual run

In [5]:
flt = [T.USDC, T.USDT, T.ETH, T.LINK, T.BNT, T.WBTC, T.DAI]
# bot._run(flashloan_tokens=flt, CCm=CCm2)

## Analytics

In [6]:
ops = bot._find_arbitrage_opportunities_carbon_single_pairwise(flashloan_tokens=flt, CCm=CCm2, result=bot.AO_CANDIDATES)
ops

[(Decimal('0.416954878170092166787079010988634024392762711386239971034228801727294921875'),
                                              USDT-1ec7     WETH-6Cc2
  1f595b1af09543b6959e9f1cb23b929d             4.804612 -2.644104e-03
  2722258935367507707706996859454145691656-0  -5.000000  2.644104e-03
  AMMIn                                        4.804612  2.644104e-03
  AMMOut                                      -5.000000 -2.644104e-03
  TOTAL NET                                   -0.195388  1.818989e-11,
  [{'cid': '1f595b1af09543b6959e9f1cb23b929d',
    'tknin': 'USDT-1ec7',
    'amtin': 4.80461186170578,
    'tknout': 'WETH-6Cc2',
    'amtout': -0.002644103631610051,
    'error': None},
   {'cid': '2722258935367507707706996859454145691656-0',
    'tknin': 'WETH-6Cc2',
    'amtin': 0.002644103649799945,
    'tknout': 'USDT-1ec7',
    'amtout': -5.0,
    'error': None}],
  'USDT-1ec7',
  (CPCArbOptimizer.TradeInstruction(cid='1f595b1af09543b6959e9f1cb23b929d', tknin='USDT-1ec7', amt

In [7]:
# ord = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ORDSCAL)
# ord

In [8]:
# ti = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_TI)
# ti

In [9]:
# ti

In [10]:
# ops = bot._find_arbitrage_opportunities(flashloan_tokens=flt, CCm=CCm, result=bot.AO_CANDIDATES)
# ops

In [11]:
candf = pd.DataFrame(ops).sort_values(by=0, ascending=False)
candf.reset_index(inplace=True, drop=True)
candf.iloc[0][2]

[{'cid': '8748e770adfe49f78064fd9bcf72417e',
  'tknin': 'USDC-eB48',
  'amtin': 494.26729866862297,
  'tknout': 'WBTC-C599',
  'amtout': -0.018035669663731824,
  'error': None},
 {'cid': '4083388403051261561560495289181218537493-0',
  'tknin': 'WBTC-C599',
  'amtin': 0.018035669672296306,
  'tknout': 'USDC-eB48',
  'amtout': -499.6051326568995,
  'error': None}]

In [12]:
# print(CCm.bycid('340282366920938463463374607431768211527-1').description)

In [13]:
candf.iloc[0][0]

Decimal('11.3842947900548393942068657880771156162244854212484135302929644240066409111023')

In [14]:
exact = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_EXACT)
exact

[TradeInstruction(ConfigObj=Config(network=_ConfigNetworkMainnet(), db=_ConfigDBPostgres(), logger=_ConfigLoggerDefault(), provider=_ConfigProviderAlchemy()), cid='8748e770adfe49f78064fd9bcf72417e', tknin='USDC-eB48', amtin=Decimal('493.773031'), tknout='WBTC-C599', amtout=Decimal('0.01796178'), pair_sorting='', raw_txs='[]', custom_data='', db=DatabaseManager(ConfigObj=Config(network=_ConfigNetworkMainnet(), db=_ConfigDBPostgres(), logger=_ConfigLoggerDefault(), provider=_ConfigProviderAlchemy()), contracts={}, filters=[], data=                                       cid                last_updated  \
 0     00125d26-4f9d-4936-9a46-7e7708cee9b5  2023-05-06 19:04:10.202420   
 1219  5bbd01d2-ba90-49e8-8235-0f5183a44204  2023-05-06 19:11:11.844129   
 1231  5c976f61-5847-4219-a2ee-0b1eb96b2775  2023-05-06 19:18:59.176213   
 1230  5c87da80-15a0-4c59-baac-e8e9661ee920  2023-05-06 19:00:47.931782   
 2635  c21eb1b9-cf04-4ec5-bad1-77fb1b29b38b  2023-05-06 19:13:37.076908   
 ...            

In [15]:
exact[0].amtin_wei, exact[0].amtout_wei

(493773031, 1796178)

In [16]:
exact[1].amtin_wei, exact[1].amtout_wei

(1796178, 496535272)

In [17]:
best_profit = exact[-1].amtout - exact[0].amtin
print(best_profit)

2.762241


In [18]:
route_struct, flashloan_amount, flashloan_token_address = bot._run(flashloan_tokens=flt, CCm=CCm2, result=bot.XS_ROUTE)

2023-05-11 11:11:02,282 [fastlane:INFO] - Opportunity with profit: 5.8911846816969054028643171250223531387746334075927734375 does not meet minimum profit: 60, discarding.


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
route_struct, flashloan_amount, flashloan_token_address

# Tenderly Tx Submission

In [ ]:
# C.w3.provider.endpoint_uri

In [ ]:
# C.w3.eth.getBlock(C.w3.eth.block_number).timestamp

In [ ]:
# C.provider.CARBON_CONTROLLER_CONTRACT.strategy(340282366920938463463374607431768211527)

In [ ]:
### This to manually submit
# bot._validate_and_submit_transaction_tenderly(
#         ConfigObj = C,
#         route_struct = route_struct,
#         src_amount = flashloan_amount,
#         src_address = flashloan_token_address,
#             )

# MAINNNET Preferred Tx Submission Method

In [ ]:
from fastlane_bot.helpers import TxHelpers

In [ ]:



pool = (
    bot.db.get_pool(exchange_name=bot.ConfigObj.BANCOR_V3_NAME, pair_name='BNT-FF1C/ETH-EEeE')
)
bnt_eth = (int(pool.tkn0_balance), int(pool.tkn1_balance))

# Init TxHelpers
tx_helpers = TxHelpers(ConfigObj=bot.ConfigObj)
# Submit tx

current_gas_price, current_max_priority_gas, block_number, nonce = tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
                                                    src_address=flashloan_token_address, bnt_eth=bnt_eth,
                                                    expected_profit=Decimal('1000'), safety_override=True, result=tx_helpers.XS_API_CALLS)

# print(current_gas_price, current_max_priority_gas, block_number, nonce)

#print()

tx = tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
                                                    src_address=flashloan_token_address, bnt_eth=bnt_eth,
                                                    expected_profit=Decimal('1000'), safety_override=False, result=tx_helpers.XS_TRANSACTION)

print(tx)

In [ ]:
# tx_helpers.validate_and_submit_transaction(route_struct=route_struct, src_amt=flashloan_amount,
#                                                     src_address=flashloan_token_address, bnt_eth=bnt_eth,
#                                                     expected_profit=Decimal('1000'), safety_override=True)

# MAINNET Forced Tx Submission Method

In [ ]:
# # Initialize tx helper
# tx_helper = TxHelper(
#     usd_gas_limit=55,
#     ConfigObj=bot.ConfigObj,
#     gas_price_multiplier = 1
# )
# # # Submit the transaction
# # tx_helper.submit_flashloan_arb_tx(
# #     arb_data=route_struct,
# #     flashloan_token_address=flashloan_token_address,
# #     flashloan_amount=flashloan_amount,
# #     verbose=True,
# # )

In [ ]:
# tx_helper.gas_limit